In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import pypsa
import yaml

In [3]:
TY = 2030

simulation_year = 2010

bus_locations = pd.read_csv('../resources/bus_locations.csv', header=0, index_col=0)

basedir = f'/srv/ES2050/ensym/DestinE/ERAA/scenario_{TY}_{simulation_year}/'

In [4]:
with open('../config.yaml', 'r') as excelspecsfile:
        config = yaml.safe_load(excelspecsfile)

In [62]:
### read net transfer capacities, specify links
network = pypsa.Network(name='DestinE Pilot base')
network.madd('Bus', bus_locations.index, x=bus_locations.x, y=bus_locations.y)

for link_type in config['transmission']['link_types'][:1]:

    network.add('Carrier', link_type)

    link_specs = pd.read_csv(basedir+f'links_technical_{link_type}_{TY}.csv', header=0, index_col=0)

    link_p_max_pu = pd.read_csv(basedir+f'links_cf_timeseries_{link_type}_{TY}.csv', header=0, index_col=0).reindex(columns=link_specs.index, fill_value=1.)
    link_p_min_pu = -link_p_max_pu[link_specs.index[link_specs.bidirectional]]
    
    snapshots = pd.DatetimeIndex(pd.date_range(start=f'{simulation_year}-01-01 00:00:00', freq='H', periods=link_p_max_pu.shape[0], name='snapshots'))
    link_p_max_pu.index = snapshots
    link_p_min_pu.index = snapshots

    link_specs.index = [f'{ll} {link_type}' for ll in link_specs.index]
    link_p_max_pu.columns = [f'{ll} {link_type}' for ll in link_p_max_pu.columns]
    link_p_min_pu.columns = [f'{ll} {link_type}' for ll in link_p_min_pu.columns]

    if link_type == config['transmission']['link_types'][0]:
        network.set_snapshots(snapshots)

    network.madd('Link', 
         names, 
         bus0=link_specs['start'], 
         bus1=link_specs['end'], 
         p_min_pu=link_p_min_pu, 
         p_max_pu=link_p_max_pu, 
         p_nom=link_specs['nominal_capacity'], 
         carrier=link_type
    )

In [5]:
### identify links with constant p_min_pu/p_max_pu, drop from time series dataframe
constant_p_min_pu = network.links.index[network.links_t.p_min_pu.min() == network.links_t.p_min_pu.max()]
constant_p_max_pu = network.links.index[network.links_t.p_max_pu.min() == network.links_t.p_max_pu.max()]

network.links.loc[constant_p_min_pu, 'p_min_pu'] = network.links_t.p_min_pu.min()[constant_p_min_pu]
network.links_t.p_min_pu.drop(constant_p_min_pu, axis=1, inplace=True)
network.links.loc[constant_p_max_pu, 'p_max_pu'] = network.links_t.p_max_pu.max()[constant_p_max_pu]
network.links_t.p_max_pu.drop(constant_p_max_pu, axis=1, inplace=True)

In [8]:
### identify duplicated links (those from 'hidden columns' in excel sheet) and drop
duplicated_links = network.links.index[[len(l.split('.'))==2 for l in network.links.index]]
network.mremove('Link', duplicated_links)

In [9]:
network.export_to_netcdf(f'../networks/pilot_base_TY{TY}_{simulation_year}.nc')

INFO:pypsa.io:Exported network pilot_base_TY2030_2010.nc has links, buses


<xarray.Dataset>
Dimensions:                       (snapshots: 8760, investment_periods: 0,
                                   links_i: 227, links_t_p_min_pu_i: 12,
                                   links_t_p_max_pu_i: 18, buses_i: 60)
Coordinates:
  * snapshots                     (snapshots) int64 0 1 2 3 ... 8757 8758 8759
  * investment_periods            (investment_periods) int64 
  * links_i                       (links_i) object 'AL00-GR00 HVAC' ... 'UK00...
  * links_t_p_min_pu_i            (links_t_p_min_pu_i) object 'BE00-LUB1 HVAC...
  * links_t_p_max_pu_i            (links_t_p_max_pu_i) object 'DE00-DKW1 HVAC...
  * buses_i                       (buses_i) object 'AL00' 'AT00' ... 'UKNI'
Data variables: (12/16)
    snapshots_snapshot            (snapshots) datetime64[ns] 2010-01-01 ... 2...
    snapshots_objective           (snapshots) float64 1.0 1.0 1.0 ... 1.0 1.0
    snapshots_stores              (snapshots) float64 1.0 1.0 1.0 ... 1.0 1.0
    snapshots_generators          (snapshots) float64 1.0 1.0 1.0 ... 1.0 1.0
    investment_periods_objective  (investment_periods) object 
    investment_periods_years      (investment_periods) object 
    ...                            ...
    links_p_min_pu                (links_i) float64 0.0 0.0 0.0 ... -1.0 0.0 0.0
    links_p_max_pu                (links_i) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    links_t_p_min_pu              (snapshots, links_t_p_min_pu_i) float64 -1....
    links_t_p_max_pu              (snapshots, links_t_p_max_pu_i) float64 1.0...
    buses_x                       (buses_i) float64 19.91 14.27 ... -6.948
    buses_y                       (buses_i) float64 41.25 47.42 ... 53.26 54.65
Attributes:
    network_name:           DestinE Pilot base
    network_pypsa_version:  0.23.0
    network_srid:           4326
    meta:                   {}